# 데이터 로드 및 확인

In [ ]:
import json
import re

# 전처리 추가 (시스템 태그 삭제)
def clean_text(text):
    # 시스템 태그 제거
    text = re.sub(r"#@[^#]+#", "", text)
    # 중복된 기호, 공백 정리
    text = re.sub(r"[!@#\$%^&*\(\)\[\]_+=<>?/|\\~`\"';:]{2,}", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()


def preprocess_dialogues(json_path, save_path=None):
    import pandas as pd

    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    pairs = []

    for dialogue in data['data']:
        body = dialogue['body']
        body.sort(key=lambda x: (x['turnID'], x['utteranceID']))
        
        prev_participant = None
        prev_text = ""
        
        for utt in body:
            pid = utt['participantID']
            text = clean_text(utt['utterance'])
            
            if not text:
                continue
            
            if prev_participant and pid != prev_participant:
                # 서로 다른 참여자 간 대화
                pairs.append((prev_text, text))
            
            prev_participant = pid
            prev_text = text
        

    if save_path:
        df = pd.DataFrame(pairs, columns=["input", "response"])
        df.to_csv(save_path, index=False, encoding='utf-8-sig')

    return pairs

In [11]:
# 전처리 실행
train_pairs = preprocess_dialogues("../data/text_dataset/한국어SNS_train/[라벨]한국어SNS_train/개인및관계.json", save_path="../data/text_dataset/save_path/train_pairs.csv")
valid_pairs = preprocess_dialogues("../data/text_dataset/한국어SNS_valid/[라벨]한국어SNS_valid/개인및관계.json", save_path="../data/text_dataset/save_path/valid_pairs.csv")

Q: 잉ㅜㅜ
A: 돈따스
------------------------------
Q: 돈따스
A: 안보내줫어?
------------------------------
Q: 이거
A: 하 ......ㅡ
------------------------------
Q: 퀵으로한대서 두시까지오래 ㅋㅋㅋㅋ
A: ㅎㅎㅎㅎ오좋겠네
------------------------------
Q: ㅎㅎㅎㅎ오좋겠네
A: 잘잣어ㅋㅋㅋㅋㅋ
------------------------------


In [13]:
# 학습 샘플 확인
for i in range(20):
    print(f"Q: {train_pairs[i][0]}")
    print(f"A: {train_pairs[i][1]}")
    print("-" * 30)

Q: 잉ㅜㅜ
A: 돈따스
------------------------------
Q: 돈따스
A: 안보내줫어?
------------------------------
Q: 이거
A: 하 ......ㅡ
------------------------------
Q: 퀵으로한대서 두시까지오래 ㅋㅋㅋㅋ
A: ㅎㅎㅎㅎ오좋겠네
------------------------------
Q: ㅎㅎㅎㅎ오좋겠네
A: 잘잣어ㅋㅋㅋㅋㅋ
------------------------------
Q: 잘잣어ㅋㅋㅋㅋㅋ
A: ㅋㄱㅋㄱㄱㄱㄱ아니
------------------------------
Q: 머거
A: 잉
------------------------------
Q: 내돈가쓰...
A: ㅋㄱㄱㄱㄱㄱ맛있어
------------------------------
Q: 고로케도존맛탱
A: 사진찍엇어....
------------------------------
Q: 사진찍엇어....
A: ㅋㄱㄱㄱㅋ
------------------------------
Q: 학생이면좋구!
A: 훔
------------------------------
Q: 없는데...주변에...
A: 왜혼자다니냐고오.....
------------------------------
Q: 왜혼자다니냐고오.....
A: 아니
------------------------------
Q: 어케 친구가있냐..
A: 와 내친군학교나감
------------------------------
Q: 막졸업한애두굳
A: 없다구...
------------------------------
Q: 너무화난당..
A: 흠
------------------------------
Q: 흠
A: 근데오빠는말을또 잘해서 내가화내다보면결국내잘못
------------------------------
Q: 답답해진짱ㅋㅋ
A: 그럴때 억울하지 짖짜
------------------------------
Q: 오빠도 오늘 회식이야?
A: 아니
----------

----

# 